Imports

In [1]:
import f90nml
import numpy as np
from pint import UnitRegistry; AssignQuantity = UnitRegistry().Quantity
import os
import reference_solution as refsol
from scipy.fft import rfft
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import icepinn as ip

torch.set_default_dtype(torch.float64)
print(torch.cuda.device_count())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

device = ip.get_device()

cuda
1
cuda


In [2]:
# Read in GI parameters
inputfile = "GI parameters - Reference limit cycle (for testing).nml"
GI=f90nml.read(inputfile)['GI']
nx_crystal = GI['nx_crystal']
L = GI['L']
NBAR = GI['Nbar']
NSTAR = GI['Nstar']

# Define t range (needs to be same as training file)
RUNTIME = 5
NUM_T_STEPS = RUNTIME + 1
#NUM_T_STEPS = RUNTIME*5 + 1

# Define initial conditions
Ntot_init = np.ones(nx_crystal)
Nqll_init = ip.get_Nqll(Ntot_init)

# Define x, t pairs for training
X_QLC = np.linspace(-L,L,nx_crystal)
t_points = np.linspace(0, RUNTIME, NUM_T_STEPS)
x, t = np.meshgrid(X_QLC, t_points)
training_set = torch.tensor(np.column_stack((x.flatten(), t.flatten()))).to(device)

In [3]:
# Define model attributes; instantiate model
model_dimensions = torch.tensor([8, 16]).to(device)
is_sf_PINN = torch.tensor(False)
model1 = ip.IcePINN(
	num_hidden_layers=model_dimensions[0], 
	hidden_layer_size=model_dimensions[1],
	is_sf_PINN=is_sf_PINN.item()).to(device)

# Attach model attributes as buffers so they can be saved and loaded
model1.register_buffer('dimensions', model_dimensions)
model1.register_buffer('is_sf_PINN', is_sf_PINN)

# Initialize model weights with HE initialization
model1.apply(ip.init_HE)

optimizer = torch.optim.AdamW(model1.parameters(), lr=1e-5)

# Define learning rate scheduling scheme
scheduler_summed = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=10000
    )

In [4]:
MODEL_NAME = "HE_16wide_nodiff"
print(MODEL_NAME)
print(training_set.shape)
print(ip.calc_cp_loss(model1, training_set, ip.get_misc_params(), 10, 1, 1, False, False).shape)
print(ip.enforced_model(training_set, model1).shape)
print(model1)

HE_16wide_nodiff
torch.Size([1920, 2])
torch.Size([1920, 2])
torch.Size([1920, 2])
IcePINN(
  (sml): SinusoidalMappingLayer()
  (post_sml): Linear(in_features=48, out_features=16, bias=True)
  (sin): SinActivation()
  (fc_in): Linear(in_features=2, out_features=16, bias=True)
  (post_fc_in): Linear(in_features=16, out_features=16, bias=True)
  (fc_hidden): ModuleList(
    (0-5): 6 x Linear(in_features=16, out_features=16, bias=True)
  )
  (fc_out): Linear(in_features=16, out_features=2, bias=True)
)


Train model

In [6]:
ip.train_IcePINN(
    model=model1, 
    optimizer=optimizer, 
    training_set=training_set, 
    epochs=100_000, 
    name=MODEL_NAME, 
    print_every=1_000, 
    print_gradients=False,
    diffusion=False,
    LR_scheduler=None)

Commencing PINN training on 1920 points for 100000 epochs.
Epoch [1k/100k] at 0m 10s: Ntot = 988.172, Nqll = 173.996, LR = 1e-05
Epoch [2k/100k] at 0m 20s: Ntot = 1010.521, Nqll = 164.417, LR = 1e-05
Epoch [3k/100k] at 0m 29s: Ntot = 1025.103, Nqll = 160.719, LR = 1e-05
Epoch [4k/100k] at 0m 37s: Ntot = 1034.273, Nqll = 160.441, LR = 1e-05
Epoch [5k/100k] at 0m 44s: Ntot = 1052.674, Nqll = 164.185, LR = 1e-05
Epoch [6k/100k] at 0m 50s: Ntot = 1080.910, Nqll = 172.616, LR = 1e-05
Epoch [7k/100k] at 0m 56s: Ntot = 1095.320, Nqll = 180.925, LR = 1e-05
Epoch [8k/100k] at 1m 2s: Ntot = 1084.001, Nqll = 182.407, LR = 1e-05
Epoch [9k/100k] at 1m 10s: Ntot = 1034.580, Nqll = 173.525, LR = 1e-05
Epoch [10k/100k] at 1m 17s: Ntot = 995.731, Nqll = 163.079, LR = 1e-05
Epoch [11k/100k] at 1m 25s: Ntot = 971.013, Nqll = 157.215, LR = 1e-05
Epoch [12k/100k] at 1m 36s: Ntot = 936.200, Nqll = 149.711, LR = 1e-05
Epoch [13k/100k] at 1m 46s: Ntot = 892.919, Nqll = 140.114, LR = 1e-05
Epoch [14k/100k] at 

In [ ]:
# Instantiate model2 as best version of model1
# model2 = ip.load_IcePINN(MODEL_NAME)
# model2.train()
# optimizer2 = torch.optim.AdamW(model2.parameters(), lr=1e-6)

Train again on smaller LR starting from saved checkpoint (best saved model above)

In [ ]:
# ip.train_IcePINN(
#     model=model2, 
#     optimizer=optimizer2, 
#     training_set=training_set, 
#     epochs=100_000, 
#     name=MODEL_NAME+"_round2", 
#     print_every=1_000, 
#     print_gradients=False,
#     diffusion=True,
#     LR_scheduler=None)